# Rock Paper Scissors
This is my solution for the Rock Paper Scissors project from FreeCodeCamp. Instructions for this project can be found [here](https://www.freecodecamp.org/learn/machine-learning-with-python/machine-learning-with-python-projects/rock-paper-scissors).

## Introduction
It took me a while to crack it up. After exploring several different approaches, I found that Markov chains provided the best solution.

With the solution code, I also copied all the challenge test code into separated cells.

The original upload on Replit with my previous attempts can be found [here](https://replit.com/@yemelgen/boilerplate-rock-paper-scissors).

In [1]:
import numpy as np

# Possible plays (moves)
plays = ('R', 'P', 'S')

# Possible combinations of plays (outcomes)
outcomes = ('RR', 'RP', 'RS', 'PR', 'PP', 'PS', 'SR', 'SP', 'SS' )

# Trainsition matrix
transitions = np.ones( (9, 3) )

# Best play based on the current guess
best_play = lambda x: (x + 1) % 3

def player( prev_play, history=[] ):
    global transitions
    guess = np.random.choice( (0, 1, 2) ) # random prediciton

    if not prev_play or not history:
      history.clear()
      next_play = plays[ best_play(guess) ]
      history.append( [next_play] )
      return next_play
    history[-1].append( prev_play )
    if len(history) >= 2:
      # Create indices of the last and previous outcomes
      last_outcome = outcomes.index( history[-1][0] + history[-1][1] )
      prev_outcome = outcomes.index( history[-2][0] + history[-2][1] )

      # Update the transition matrix by decaying earlier observations and incrementing the count of the previous play
      transitions[ prev_outcome ] *= 0.8 # decay rate
      transitions[ prev_outcome, plays.index(prev_play) ] += 1

      # Check if the values in the transition matrix of the last outcome are not equal
      if np.ptp( transitions[last_outcome] ) != 0:
        # Find the play with the highest value in the transition matrix for the last outcome
        guess = np.argmax( transitions[last_outcome] )
    next_play = plays[ best_play(guess) ]
    history.append( [next_play] )

    return next_play

In [2]:
# NOTE: Copied from RPS_Game.py. 
# DO NOT MODIFY THIS CELL
import random

def play(player1, player2, num_games, verbose=False):
    p1_prev_play = ""
    p2_prev_play = ""
    results = {"p1": 0, "p2": 0, "tie": 0}

    for _ in range(num_games):
        p1_play = player1(p2_prev_play)
        p2_play = player2(p1_prev_play)

        if p1_play == p2_play:
            results["tie"] += 1
            winner = "Tie."
        elif (p1_play == "P" and p2_play == "R") or (
                p1_play == "R" and p2_play == "S") or (p1_play == "S"
                                                       and p2_play == "P"):
            results["p1"] += 1
            winner = "Player 1 wins."
        elif p2_play == "P" and p1_play == "R" or p2_play == "R" and p1_play == "S" or p2_play == "S" and p1_play == "P":
            results["p2"] += 1
            winner = "Player 2 wins."

        if verbose:
            print("Player 1:", p1_play, "| Player 2:", p2_play)
            print(winner)
            print()

        p1_prev_play = p1_play
        p2_prev_play = p2_play

    games_won = results['p2'] + results['p1']

    if games_won == 0:
        win_rate = 0
    else:
        win_rate = results['p1'] / games_won * 100

    print("Final results:", results)
    print(f"Player 1 win rate: {win_rate}%")

    return (win_rate)


def quincy(prev_play, counter=[0]):

    counter[0] += 1
    choices = ["R", "R", "P", "P", "S"]
    return choices[counter[0] % len(choices)]


def mrugesh(prev_opponent_play, opponent_history=[]):
    opponent_history.append(prev_opponent_play)
    last_ten = opponent_history[-10:]
    most_frequent = max(set(last_ten), key=last_ten.count)

    if most_frequent == '':
        most_frequent = "S"

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[most_frequent]


def kris(prev_opponent_play):
    if prev_opponent_play == '':
        prev_opponent_play = "R"
    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prev_opponent_play]


def abbey(prev_opponent_play,
          opponent_history=[],
          play_order=[{
              "RR": 0,
              "RP": 0,
              "RS": 0,
              "PR": 0,
              "PP": 0,
              "PS": 0,
              "SR": 0,
              "SP": 0,
              "SS": 0,
          }]):

    if not prev_opponent_play:
        prev_opponent_play = 'R'
    opponent_history.append(prev_opponent_play)

    last_two = "".join(opponent_history[-2:])
    if len(last_two) == 2:
        play_order[0][last_two] += 1

    potential_plays = [
        prev_opponent_play + "R",
        prev_opponent_play + "P",
        prev_opponent_play + "S",
    ]

    sub_order = {
        k: play_order[0][k]
        for k in potential_plays if k in play_order[0]
    }

    prediction = max(sub_order, key=sub_order.get)[-1:]

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prediction]


def human(prev_opponent_play):
    play = ""
    while play not in ['R', 'P', 'S']:
        play = input("[R]ock, [P]aper, [S]cissors? ")
        print(play)
    return play


def random_player(prev_opponent_play):
    return random.choice(['R', 'P', 'S'])

In [3]:
# NOTE: Copied from test_module.py
import unittest

class UnitTests(unittest.TestCase):
    print()

    def test_player_vs_quincy(self):
        print("Testing game against quincy...")
        actual = play(player, quincy, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat quincy at least 60% of the time.')

    def test_player_vs_abbey(self):
        print("Testing game against abbey...")
        actual = play(player, abbey, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat abbey at least 60% of the time.')

    def test_player_vs_kris(self):
        print("Testing game against kris...")
        actual = play(player, kris, 1000) >= 60
        self.assertTrue(
            actual, 'Expected player to defeat kris at least 60% of the time.')

    def test_player_vs_mrugesh(self):
        print("Testing game against mrugesh...")
        actual = play(player, mrugesh, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat mrugesh at least 60% of the time.')

In [4]:
# NOTE: Copied from main.py
play(player, quincy, 1000)
play(player, abbey, 1000)
play(player, kris, 1000)
play(player, mrugesh, 1000)
unittest.main(argv=[''], verbosity=2, exit=False)

test_player_vs_abbey (__main__.UnitTests) ... ok
test_player_vs_kris (__main__.UnitTests) ... 

Final results: {'p1': 744, 'p2': 101, 'tie': 155}
Player 1 win rate: 88.04733727810651%
Final results: {'p1': 652, 'p2': 315, 'tie': 33}
Player 1 win rate: 67.42502585315408%
Final results: {'p1': 992, 'p2': 6, 'tie': 2}
Player 1 win rate: 99.39879759519037%
Final results: {'p1': 817, 'p2': 154, 'tie': 29}
Player 1 win rate: 84.14006179196704%
Testing game against abbey...
Final results: {'p1': 674, 'p2': 319, 'tie': 7}
Player 1 win rate: 67.87512588116817%
Testing game against kris...


ok
test_player_vs_mrugesh (__main__.UnitTests) ... ok
test_player_vs_quincy (__main__.UnitTests) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.270s

OK


Final results: {'p1': 986, 'p2': 7, 'tie': 7}
Player 1 win rate: 99.29506545820746%
Testing game against mrugesh...
Final results: {'p1': 826, 'p2': 153, 'tie': 21}
Player 1 win rate: 84.37180796731359%
Testing game against quincy...
Final results: {'p1': 734, 'p2': 103, 'tie': 163}
Player 1 win rate: 87.69414575866189%
